In [322]:
import os
import json
import boto3
import pickle
import requests
import implicit
import numpy as np
import pandas as pd

from datetime import datetime
from scipy.sparse import csr_matrix
from sklearn.preprocessing import normalize

In [359]:
def validate_storage(bucket):
    try:
        config = get_json(bucket, 'config.json')
    except:
        config = {
            "LastModified": None, 
            "ActualModel": None, 
            "ActualScore": None, 
            "IsActual": False
        }
        set_json(config, bucket, 'config.json')
    try:
        models = get_json(bucket, 'models.json')
    except:
        models = {
            "models":[
            ]
        }
        set_json(models, bucket, 'models.json')

def check_for_updates(bucket):
    config = get_json(bucket, 'config.json')
    last_update_date = max([
            get_file_last_update_date(bucket, 'data/raw/lastfm_artist_list.csv'), 
            get_file_last_update_date(bucket, 'data/raw/lastfm_user_scrobbles.csv')
        ]
    ).strftime('%m.%d.%Y_%H:%M:%S')
    if config['LastModified'] != last_update_date:
        config['LastModified'] = last_update_date
        config['IsActual'] = False
        set_json(config, bucket, 'config.json')

def process_scrobbles(config, bucket):
    scrobbles = get_csv(bucket, 'data/raw/lastfm_user_scrobbles.csv')
    scrobbles.drop_duplicates(['user_id', 'artist_id'], inplace=True)
    users_indexes, users_positions = np.unique(scrobbles.values[:,0], return_inverse=True)
    artists_indexes, artists_positions = np.unique(scrobbles.values[:,1], return_inverse=True)
    scrobbles_sparse = csr_matrix((scrobbles.values[:,2], (users_positions, artists_positions)))
    scrobbles_sparse_normalized = normalize(scrobbles_sparse, norm='l2', axis=1)
    pickle.dump(scrobbles.groupby('user_id')['artist_id'].apply(list).to_dict(), open(f"data/{config['LastModified']}/scrobbles.pkl", 'wb'))
    pickle.dump(scrobbles_sparse_normalized, open(f"data/{config['LastModified']}/scrobbles_sparse_normalized.pkl", 'wb'))
def upload_scrobbles(config, bucket):
    upload_file(bucket, f"data/{config['LastModified']}/scrobbles.pkl", f"data/{config['LastModified']}/scrobbles.pkl")
    os.remove(f"data/{config['LastModified']}/scrobbles.pkl")
    upload_file(bucket, f"data/{config['LastModified']}/scrobbles_sparse_normalized.pkl", f"data/{config['LastModified']}/scrobbles_sparse_normalized.pkl")
    os.remove(f"data/{config['LastModified']}/scrobbles_sparse_normalized.pkl")
def process_data(bucket):
    config = get_json(bucket, 'config.json')
    if config['IsActual'] == False:
        os.makedirs(f"data/{config['LastModified']}/", exist_ok=True)
        process_scrobbles(config, bucket)
        upload_scrobbles(config, bucket)
        os.rmdir(f"data/{config['LastModified']}")    

def train(bucket, name):
    config = get_json(bucket, 'config.json')
    os.makedirs(f"data/actual/", exist_ok=True)
    pickle.dump(get_pickle(bucket, f"data/{config['LastModified']}/scrobbles_sparse_normalized.pkl"), open('data/actual/scrobbles_sparse_normalized.pkl', 'wb'))
    model = eval(name)
    model.fit(pickle.load(open('data/actual/scrobbles_sparse_normalized.pkl', 'rb')))
    pickle.dump(model, open(f"models/{name}.pkl", 'wb'))
    upload_file(bucket, f"models/{name}.pkl", f"models/{name}.pkl")
    config = get_json(bucket, 'config.json')
    config['IsActual'] = False
    set_json(config, bucket, 'config.json')

def get_k_similar(model, target, k):
    indices, distances = model.similar_items(target-1, N=5, filter_items=[target-1])
    return [x+1 for x in indices]
def evaluate(bucket, name):
    config = get_json(bucket, 'config.json')
    os.makedirs(f"data/actual/", exist_ok=True)
    pickle.dump(get_pickle(bucket, f"data/{config['LastModified']}/scrobbles.pkl"), open('data/actual/scrobbles.pkl', 'wb'))
    validation = pickle.load(open('data/actual/scrobbles.pkl', 'rb'))
    
    model = pickle.load(open(f"models/{name}.pkl", 'rb'))
    models = get_json(bucket, 'models.json')
    scores = []
    users = 0
    for user in validation:
        if len(validation[user]) <= 1:
            continue
        predictions = []
        for artist in validation[user]:
            predictions += get_k_similar(model, artist, 5)
        scores.append(precision(validation[user], predictions))
        users += 1
    score = mean_precision(scores, users)
    now = datetime.now().strftime('%m.%d.%Y_%H:%M:%S')
    models['models'].append({
        "name":name,
        "score":score,
        "date":now
    })
    upload_file(bucket, f"models/{name}.pkl", f"models/{name}.pkl")
    set_json(models, bucket, 'models.json')

def select_best_model(bucket):
    config = get_json(bucket, 'config.json')
    models = get_json(bucket, 'models.json')
    best_score = 0
    best_model = ''
    for model in models['models']:
        if model['date'] > config['LastModified']:
            if model['score'] > best_score:
                best_score = model['score']
                best_model = model['name']
    config['ActualModel'] = best_model
    config['ActualScore'] = best_score
    config['IsActual'] = True
    set_json(config, bucket, 'config.json')

def update_server():
    requests.post(f"http://http://87.239.106.15:8080/update")

In [360]:
bucket = 'LastFM-artist-recommender'

validate_storage(bucket)

check_for_updates(bucket)
process_data(bucket)
upload_processed_data(bucket)
load_data(bucket)

model1 = 'implicit.als.AlternatingLeastSquares(factors=64)'
train(bucket, model1)
evaluate(bucket, model1)

model2 = 'implicit.nearest_neighbours.CosineRecommender(K=200)'
train(bucket, model2)
evaluate(bucket, model2)

select_best_model(bucket)

update_server(bucket)

100%|██████████| 15/15 [00:07<00:00,  1.96it/s]
/Users/grshchnkv/opt/anaconda3/envs/LastFM-artist-recommender/lib/python3.10/site-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0011570453643798828 seconds
  warnings.warn(
100%|██████████| 17493/17493 [00:00<00:00, 191303.86it/s]
